# Soh LR estimation evaluation

The goal of this notebook is to find ways to evaluate the quality of the soh estimation.

## Setup

In [ ]:
import logging

import plotly.express as px
import pandas as pd
from pandas import DataFrame as DF
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import numpy as np

from core.plt_utils import plt_3d_df
from watea.watea_constants import *
from watea.energy_distribution import *

logging.basicConfig(level=logging.INFO)

In [ ]:
cluster = get_processed_cluster()

## Evaluate result over odometer
We will evaluate the results per vehicle and on the entire fleet.

For now we will implement a monotonicity score and a 

### Per vehicle